<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/55_Khushi/PoseStreamModel/PoseStreamModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [ ]:
import pandas as pd

# read the data from the CSV file
data = pd.read_csv('train_data.csv')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

In [ ]:
numOfKeypoints = 34
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model


In [ ]:
type(train_keypoints[0][0][1][0])

NoneType

In [ ]:
# calculate the center of the object in each frame
centers = []
for keypoints in train_keypoints:
    frame_centers = []
    for frame_keypoints in keypoints:
        # assume the center is the mean of all keypoints
       center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
       center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
       frame_centers.append((center_x, center_y))
    centers.append(frame_centers)

train_relative_keypoints = train_keypoints
# make the keypoints relative to the center
for i in range(len(train_relative_keypoints)):
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is not None:
               train_relative_keypoints[i][j][k][0] -= centers[i][j][0]
            if train_relative_keypoints[i][j][k][1] is not None:
              train_relative_keypoints[i][j][k][1] -= centers[i][j][1]


<ipython-input-17-a6c48355f099>:7: RuntimeWarning: Mean of empty slice
  center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
<ipython-input-17-a6c48355f099>:8: RuntimeWarning: Mean of empty slice
  center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])


In [ ]:
count=0
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_relative_keypoints)):
    # flag=1
    frames_keypoints=[] 
    for j in range(len(train_relative_keypoints[i])):
        flag=1
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is None or train_relative_keypoints[i][j][k][1] is None:
              count+=1
              flag=0
              break
        if(flag==1):
          frames_keypoints.append(train_relative_keypoints[i][j])
    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for x in range (16-len(frames_keypoints)):
          frames_keypoints.append(kk)
      # if(len(frames_keypoints)!=16):
      #   print("2 : ",len(frames_keypoints))
      new_train_keypoints.append(frames_keypoints)
      new_train_labels.append(train_labels[i])

In [ ]:
print(len(new_train_keypoints),len(new_train_labels))

1033 1033


In [ ]:
# train_videos = np.asarray(train_videos)
new_train_keypoints = np.asarray(new_train_keypoints)
new_train_labels = np.asarray(new_train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# train_keypoints = np.array([np.array(val) for val in train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(new_train_keypoints,new_train_labels,test_size=0.25,shuffle=True)

In [ ]:
print(len(XKey_train),len(XKey_test))

774 259


In [ ]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [ ]:
# import mediapipe as mp

# landmarks = mp_pose.PoseLandmark.__members__
# print(landmarks)


In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


20/20 [==============================] - 3s 161ms/step - loss: 21.6912 - accuracy: 0.1066 - val_loss: 17.9949 - val_accuracy: 0.1419
Epoch 2/300
20/20 [==============================] - 4s 207ms/step - loss: 15.3486 - accuracy: 0.1470 - val_loss: 12.4648 - val_accuracy: 0.1548
Epoch 3/300
20/20 [==============================] - 4s 175ms/step - loss: 10.3973 - accuracy: 0.1761 - val_loss: 8.3775 - val_accuracy: 0.2258
Epoch 4/300
20/20 [==============================] - 3s 157ms/step - loss: 6.8797 - accuracy: 0.3037 - val_loss: 5.6341 - val_accuracy: 0.2774
Epoch 5/300
20/20 [==============================] - 3s 159ms/step - loss: 4.6907 - accuracy: 0.3409 - val_loss: 4.1295 - val_accuracy: 0.2903
Epoch 6/300
20/20 [==============================] - 4s 201ms/step - loss: 3.5146 - accuracy: 0.3748 - val_loss: 3.3303 - val_accuracy: 0.3677
Epoch 7/300
20/20 [==============================] - 4s 177ms/step - loss: 2.8857 - accuracy: 0.4216 - val_loss: 2.9416 - val_accuracy: 0.4065
Epoch 

In [ ]:


# Get model summary
# model.summary()

# # Get training and validation accuracy
# train_accuracy = history.history['accuracy']
# val_accuracy = history.history['val_accuracy']

# # Get training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 16, 32, 16)       112       
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 16, 16, 16)       0         
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 16, 14, 128)      6272      
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 16, 7, 128)       0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 16, 5, 50)       

In [ ]:
model.evaluate(XKey_test,yKey_test)

9/9 [==============================] - 1s 89ms/step - loss: 0.6893 - accuracy: 0.8494


[0.6892863512039185, 0.8494208455085754]

In [ ]:
# Save the model to a file
model.save('PoseStream.h5')

# Load the saved model from the file
loaded_model = load_model('PoseStream.h5')

In [ ]:
loaded_model.evaluate(XKey_test,yKey_test)

9/9 [==============================] - 1s 62ms/step - loss: 0.6893 - accuracy: 0.8494


[0.6892863512039185, 0.8494208455085754]

In [ ]:
input_data = new_train_keypoints[0]
# Reshape the input data to include the batch size dimension
input_data = np.expand_dims(input_data, axis=0)

# Make a prediction with the model
output = model.predict(input_data)
output

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 299ms/step


array([[9.99708712e-01, 2.62607158e-11, 8.06678546e-09, 1.93838652e-08,
        4.37154535e-09, 3.91757027e-07, 7.37984745e-19, 7.65237303e-07,
        2.77937128e-04, 2.42370291e-11, 1.20226105e-05, 3.20784697e-12,
        1.26892900e-07, 1.34451783e-09, 6.71775569e-13, 3.91274763e-10,
        7.01205912e-13, 3.00378666e-10, 9.27593319e-15, 4.94650870e-13]],
      dtype=float32)